## ReAct Agent using LangGraph

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name = "llama-3.3-70b-versatile")

In [4]:
llm.invoke("Hi do you think I am a good person?")

AIMessage(content='What a thoughtful and introspective question! As a conversational AI, I don\'t have personal opinions or judgments about individuals. However, I can tell you that being a good person is often subjective and can depend on various factors, such as personal values, cultural norms, and relationships.\n\nThat being said, the fact that you\'re asking this question suggests that you\'re reflecting on your own character and behavior, which is a positive sign. It implies that you\'re interested in self-improvement, empathy, and possibly making a positive impact on those around you.\n\nRather than seeking external validation, you might consider asking yourself:\n\n* What are my core values, and do my actions align with them?\n* How do I treat others, and can I be more compassionate and understanding?\n* What are my strengths and weaknesses, and how can I work on personal growth?\n\nBy exploring these questions and engaging in self-reflection, you can gain a deeper understandin

In [6]:
from langgraph.graph import MessagesState, StateGraph, END, START
from langchain_core.messages import HumanMessage, SystemMessage, AnyMessage
from langchain_community.tools import DuckDuckGoSearchResults
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display
from typing import Annotated, TypedDict
import operator
from langgraph.graph.message import add_messages


In [1]:
def multiply(a: int, b: int) -> int:
    """ Multiply the two numbers a and b and return the result """
    return a*b